# Single Frame Model

This script trains a single frame model based on an average image taken across the the forcasted month.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, Conv3D, MaxPooling2D, MaxPooling3D, LayerNormalization
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import initializers
import time
import numpy as np

## 0. Define Region

First, define a region to train a model for.

In [2]:
REGIONS = ["ES","NS","WS","EM","EE","LD","NEE","NWE","SEE","SWE","WAL","WM","YH"]

## 1. Load Data

To begin, data for a single frame mnodel must be loaded along with the expected regional rainfall values.

### 1.0 New Training Data

This section loads the new training data set.

In [4]:
training_datafile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/single_all.npy"
training_rainfallfile = "D:/PHD_DATA/Video_18-01-2021/prepared-data/expected_all.npy"

In [5]:
training_videos = np.load(training_datafile)
training_rainfall = np.load(training_rainfallfile)[:, 2:]

In [6]:
training_videos = np.swapaxes(training_videos, 1, 2)
training_videos = np.swapaxes(training_videos, 2, 3)

In [7]:
training_videos[:, :, :, 0] = (training_videos[:, :, :, 0] - np.min(training_videos[:, :, :, 0])) / (np.max(training_videos[:, :, :, 0]) - np.min(training_videos[:, :, :, 0]))
training_videos[:, :, :, 1] = (training_videos[:, :, :, 1] - np.min(training_videos[:, :, :, 1])) / (np.max(training_videos[:, :, :, 1]) - np.min(training_videos[:, :, :, 1]))

for n in range(0, len(REGIONS)):
    training_rainfall[:, n] = (training_rainfall[:, n] - np.min(training_rainfall[:, n])) / (np.max(training_rainfall[:, n]) - np.min(training_rainfall[:, n]))

In [8]:
print(np.min(training_rainfall, axis=0))
print(np.max(training_rainfall, axis=0))

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


## 2. Model Definition

Next, a CNN model architecture is defined.

In [9]:
def model_generator(input_shape=(2, 61, 121), learning_rate=0.1):
    """ This method generates a model definition. """
    model = Sequential()
    
    # First layer
    model.add(Conv2D(8, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Second layer
    model.add(Conv2D(8, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Third layer
    model.add(Conv2D(8, (2, 2), input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(LayerNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Final Layer
    model.add(Flatten())
    model.add(Dense(1))
    model.add(Activation('relu'))
    
    # Setup training mechanism
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate))
    
    return model

## 3. Model Training

Finally, training the model using the single framed data and opening a tensorboard instance with details.

In [13]:
def train_model(run_name, tensorboard, model, xdata, ydata, models_folder="D:/PHD_DATA/Video_18-01-2021/models/"):
    """ Trains the given model with the given dataset. """
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    history = model.fit(
        xdata,
        ydata,
        batch_size=8,
        validation_split=0.3,
        callbacks=[tensorboard, es],
        epochs=100
    )
    save_model(model, models_folder + run_name + ".mdl")
    return history

In [14]:
# Flexible parameters
learning_rate = 0.001

In [ ]:
final_errors = {}
# Run model for each region
for ridx, r in enumerate(REGIONS):
    final_errors[r] = {}
    # Run each model multiple times
    for i in range(0, 3):
        run_name = "{}-{}".format(r, int(time.time()))
        tb = TensorBoard(log_dir="D:/PHD_DATA/Video_11-02-2021/SingleFrame/logs/{}".format(run_name))
        model = model_generator(learning_rate=learning_rate, input_shape=training_videos.shape[1:])
        history = train_model(run_name, tb, model, training_videos, training_rainfall[:, ridx],
                              models_folder="D:/PHD_DATA/Video_11-02-2021/SingleFrame/models/")
        final_errors[r][run_name] = history.history["val_loss"][-1]

Epoch 1/100
